## Libraries

In [10]:
import json
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

## Config

In [9]:
# Load the config file
with open('../config/config.json', 'r') as f:
    config = json.load(f)

file_path = config["data_loc"]

 ## Datasets

### Inference batch

In [13]:
# Define file path
file_name = "test_unlabeled.tsv"
final_path = os.path.join(file_path, file_name) 

# Load tsv file
inference_batch = pd.read_csv(final_path, sep='\t')
print(f"The inference batch has {inference_batch.shape[0]} observations and {inference_batch.shape[1]} columns.")
inference_batch.head()

The inference batch has 1097 observations and 4 columns.


,PMID,Title,Abstract,Label
0,34902587,Detection of porcine circovirus type 3 DNA in ...,Porcine circovirus type 3 (PCV3) is regularly ...,0
1,35451025,Imputation of non-genotyped F1 dams to improve...,This study investigated using imputed genotype...,0
2,34859764,Proposed multidimensional pain outcome methodo...,Castration of male piglets in the United State...,0
3,35143972,Nanostructured lipid carriers loaded with an a...,Alopecia is a condition associated with differ...,0
4,34872491,Genome-wide expression of the residual lung re...,BACKGROUND: Acute or chronic irreversible resp...,0


### Training Corpus

In [ ]:
# Define file path
file_name = "QTL_text.json"
final_path = os.path.join(file_path, file_name) 

# Load json file
df = pd.read_json(final_path)
df = df.drop(columns=['Journal'])
print(f"Shape of the original dataset: {df.shape}", "\n")
df.head()

Shape of the original dataset: (11278, 4) 



,PMID,Title,Abstract,Category
0,17179536,Variance component analysis of quantitative tr...,"In a previous study, QTL for carcass compositi...",1
1,17177700,"Single nucleotide polymorphism identification,...",Pituitary adenylate cyclase-activating polypep...,0
2,17129674,Genetic resistance to Sarcocystis miescheriana...,Clinical and parasitological traits of Sarcocy...,0
3,17121599,Results of a whole-genome quantitative trait l...,A whole-genome quantitative trait locus (QTL) ...,1
4,17057239,Unexpected high polymorphism at the FABP4 gene...,Fatty acid bing protein 4 (FABP4) plays a key ...,0


### Pre-Processing

In [5]:
import spacy
nlp_spacy = spacy.load("en_core_web_sm")

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import string
import re

# Set stop words
stop_words = set(stopwords.words('english'))

# Set lemmatizer
lemmatizer = WordNetLemmatizer()

# Defining NLP Pre-Processing steps. These steps were the one that highlighted the most on my exploration phase
def nlp_preprocessing(abstract_tokenized):
    abstract_tokenized = abstract_tokenized.apply(lambda doc: doc.lower()) # Lowercasing txt
    abstract_tokenized = abstract_tokenized.apply(lambda doc: doc.translate(str.maketrans("", "", string.punctuation))) # Removing punctuations from the text
    # abstract_tokenized = abstract_tokenized.apply(lambda doc: re.sub(r"\d+", "", doc)) # Removing numbers from the text
    abstract_tokenized = abstract_tokenized.apply(lambda doc: [token for token in word_tokenize(doc) if token not in stop_words]) # Tokenizing and removing stop words from the text
    abstract_tokenized = abstract_tokenized.apply(lambda doc: [lemmatizer.lemmatize(token, pos=wordnet.NOUN) for token in doc]) # Convert words to their base
    abstract_tokenized = abstract_tokenized.apply(lambda doc: [token.strip() for token in doc if token.strip() and len(token)>1]) # Removing extra space
    return abstract_tokenized

[nltk_data] Downloading package punkt to /Users/gabrielvictorgomesferr
[nltk_data]     eira/opt/anaconda3/envs/nlp_env/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/gabrielvictorgomes
[nltk_data]     ferreira/opt/anaconda3/envs/nlp_env/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
# Applying pre-defined pre-processing steps
df_processed = df.copy()
df_processed['title_nltk_token'] = nlp_preprocessing(df_processed['Title'])
df_processed['abstract_nltk_token'] = nlp_preprocessing(df_processed['Abstract'])
df_processed.head()

,PMID,Title,Abstract,Category,title_nltk_token,abstract_nltk_token
0,17179536,Variance component analysis of quantitative tr...,"In a previous study, QTL for carcass compositi...",1,"[variance, component, analysis, quantitative, ...","[previous, study, qtl, carcass, composition, m..."
1,17177700,"Single nucleotide polymorphism identification,...",Pituitary adenylate cyclase-activating polypep...,0,"[single, nucleotide, polymorphism, identificat...","[pituitary, adenylate, cyclaseactivating, poly..."
2,17129674,Genetic resistance to Sarcocystis miescheriana...,Clinical and parasitological traits of Sarcocy...,0,"[genetic, resistance, sarcocystis, miescherian...","[clinical, parasitological, trait, sarcocystis..."
3,17121599,Results of a whole-genome quantitative trait l...,A whole-genome quantitative trait locus (QTL) ...,1,"[result, wholegenome, quantitative, trait, loc...","[wholegenome, quantitative, trait, locus, qtl,..."
4,17057239,Unexpected high polymorphism at the FABP4 gene...,Fatty acid bing protein 4 (FABP4) plays a key ...,0,"[unexpected, high, polymorphism, fabp4, gene, ...","[fatty, acid, bing, protein, fabp4, play, key,..."


### Key-phrase extraction

In [7]:
import gensim
from gensim.models.phrases import Phraser, Phrases

In [8]:
def gensim_key_phrase_extractor(list_of_tokens):
    bigram = Phraser(Phrases(list_of_tokens, min_count=2, threshold=15))
    bigram_token = [bigram[doc] for doc in list_of_tokens]
    return bigram_token

In [9]:
# Bigram
df_processed['title_bigram'] = gensim_key_phrase_extractor(df_processed['title_nltk_token'])
df_processed['abstract_bigram'] = gensim_key_phrase_extractor(df_processed['abstract_nltk_token'])

df_processed.head()

,PMID,Title,Abstract,Category,title_nltk_token,abstract_nltk_token,title_bigram,abstract_bigram
0,17179536,Variance component analysis of quantitative tr...,"In a previous study, QTL for carcass compositi...",1,"[variance, component, analysis, quantitative, ...","[previous, study, qtl, carcass, composition, m...","[variance_component, analysis, quantitative_tr...","[previous_study, qtl, carcass_composition, mea..."
1,17177700,"Single nucleotide polymorphism identification,...",Pituitary adenylate cyclase-activating polypep...,0,"[single, nucleotide, polymorphism, identificat...","[pituitary, adenylate, cyclaseactivating, poly...","[single_nucleotide, polymorphism, identificati...","[pituitary, adenylate, cyclaseactivating, poly..."
2,17129674,Genetic resistance to Sarcocystis miescheriana...,Clinical and parasitological traits of Sarcocy...,0,"[genetic, resistance, sarcocystis, miescherian...","[clinical, parasitological, trait, sarcocystis...","[genetic, resistance, sarcocystis, miescherian...","[clinical, parasitological, trait, sarcocystis..."
3,17121599,Results of a whole-genome quantitative trait l...,A whole-genome quantitative trait locus (QTL) ...,1,"[result, wholegenome, quantitative, trait, loc...","[wholegenome, quantitative, trait, locus, qtl,...","[result, wholegenome, quantitative_trait, locu...","[wholegenome, quantitative_trait, locus_qtl, s..."
4,17057239,Unexpected high polymorphism at the FABP4 gene...,Fatty acid bing protein 4 (FABP4) plays a key ...,0,"[unexpected, high, polymorphism, fabp4, gene, ...","[fatty, acid, bing, protein, fabp4, play, key,...","[unexpected, high, polymorphism, fabp4_gene, u...","[fatty_acid, bing, protein_fabp4, play_key, ro..."


## Modeling Exploratory

In [17]:
from sklearn.model_selection import train_test_split

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

### Train-Test Split

In [18]:
# Define predictor and target features
X = df_processed.drop(columns=['Category'])
y = df_processed['Category']

# Split train and test
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=.2, random_state=42, stratify=y)

### TF-IDF Vectors Out Of Abstracts
#### Train Set

In [33]:
train_bigram_token = X_train['abstract_bigram']
train_bigram_documents = [" ".join(token) for token in bigram_token]

# Compute TF-IDF
tfidf_vec = TfidfVectorizer(stop_words='english')
train_tfidf_matrix = tfidf_vec.fit_transform(train_bigram_documents)

print(train_tfidf_matrix.shape)
print(len(y_train))
train_tfidf_matrix[0:].toarray()

(9022, 104270)
9022


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

#### Test Set

In [34]:
val_bigram_token = X_val['abstract_bigram']
val_bigram_documents = [" ".join(token) for token in val_bigram_token]

# Compute TF-IDF
val_tfidf_matrix = tfidf_vec.transform(val_bigram_documents)

print(val_tfidf_matrix.shape)
print(len(y_val))
val_tfidf_matrix[0:].toarray()

(2256, 104270)
2256


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Train ML Models

In [35]:
from sklearn.linear_model import LogisticRegression

lg_model = LogisticRegression(solver='liblinear', random_state=42)
lg_model.fit(train_tfidf_matrix, y_train)

LogisticRegression(random_state=42, solver='liblinear')

In [36]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [37]:
# Predict on the test set:
y_pred = lg_model.predict(val_tfidf_matrix)

# Evaluate the model:
print("Accuracy:", accuracy_score(y_val, y_pred))
print("\nClassification Report:")
print(classification_report(y_val, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_val, y_pred))

Accuracy: 0.93927304964539

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      2055
           1       0.77      0.45      0.57       201

    accuracy                           0.94      2256
   macro avg       0.86      0.72      0.77      2256
weighted avg       0.93      0.94      0.93      2256


Confusion Matrix:
[[2028   27]
 [ 110   91]]
